### Cora dataset ideas:


- Feature type: Bag-of-Words (binary)

- Each node has a 1433-dimensional feature vector

- Each dimension corresponds to a unique word in the vocabulary

- Values are binary:

- 1 : the word appers in the paper

- 0 : the word does not appear

### 1. Node Classification
   - Thousands of papers are published every week —> manual categorization doesn’t scale.

  #### plan:
  - apply Shallow Embeddings with Node2Vec (only uses graph structure 'no features') to show classifiction performance only utilizing citations links
  - apply Deep Embeddings (GNNs) like GCN and GAT (makes use of node features) and perform classifiction
  - analysis (compare methods accuracies , t-SNE plots of embeddings colored by class (for each method),Discuss trade-offs (scalability, expressiveness, transductive vs. inductive))



### 2. Link Prediction
  - Authors often miss relevant prior work.
  - Citation recommendation during writing
  - Related-work discovery
  - Recommending papers that an author should read or cite based on the graph structure and/or content similarity.

  #### plan:
  - edge splitting and generating negative samples 
  - learn embeddings with link prediction objective (not sure if node2vec can use same embeddings generated for node classification or not, but for gnn we need new embeddings cause loss is different)
  - check scoring methods (For each pair (u, v) in the test set, compute a score using the embeddings using Dot product/Cosine similarity/mlp classifier )
  - evaluate predictions 